# Anzahl Features und Skalierung

Häufig bzw. *fast immer* werden Wörter oder Kombinationen von Wörtern als Features für die Vektorisierung verwendet. Allerdings wirst du merken, dass das leicht zu sehr großen Vektordimensionen führen kann.

In diesem Teil wirst du Methoden kennenlernen, wie du die Dimension des Vektorraums kontrollieren kannst.

## Daten einladen

Um die Skalierungseffekt zu sehen, arbeitest du mit der gesamten Datenmenge:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles", sql, index_col="id", parse_dates=["datePublished"])
df["full_text"] = df["title"] + "\n" + df["header"] + df["text"]

## Naiver Ansatz

Im ersten Schritt versuchst du nun, die Dokumente zu vektorisieren:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_vectors = count_vectorizer.fit_transform(df["full_text"])
count_vectors

Wie du siehst, sind das viel zu viele Features geworden.

## Auswahl der Wortarten

Prinzipiell könntest du nun die `lemmas` verwenden, was die Anzahl der Features schon beträchtlich reduziert. Du gehst aber gleich einen Schritt weiter und nutzt die Kombination aus Substantiven, Adjektiven und Verben:

In [ ]:
count_vectorizer = CountVectorizer()
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

Das ist immer noch sehr groß und sollte weiter reduziert werden.

## Stoppwörter

Stoppwörter sind Wörter, die keinen (oder wenig) Inhalt tragen. Du kennst sie noch von den Wordclouds:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
count_vectorizer = CountVectorizer(stop_words=stop_words)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

Die Anzahl der gespeicherten Elemente ist nun ganz schön gesunken, die Dimensionen hingegen haben sich nicht stark verändert.

## `min_df`

Mithilfe von `min_df` kannst du angeben, in wievielen Dokumenten Wörter mindestens vorkommen müssen, um berücksichtigt zu werden. Wenn du eine Ganzzahl angibt, ist damit die Anzahl gemeint, bei einer Kommazahl der Prozenzsatz.

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words, min_df=5)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

Das hat erhebliche Auswirkungen und die Dimension des Vektorraums erheblich reduziert!

## `max_df`

Es gibt auch noch den ergänzenden Parameter `max_df`, der Wörter ausschließen kann, die in *zu vielen Dokumenten* vorkommen. Diese haben dann ohnehin eine geringe Auszeichnungsqualität und du kannst sie weglassen. Hier ist eine Angabe als Prozentanteil fast immer sinnvoller:

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words, max_df=0.5)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

Der Parameter hat insgesamt nur einen geringen Einfluss auf die Dimension des Vektorraums.

## Anzahl der Features begrenzen

Du kannst auch einfach die Anzahl der Features begrenzen. `scikit-learn` nutzt dann einfach die häufigsten Wörter und ignoriert alle anderen. Manchmal kann das sinnvoll sein, aber zu einem Stück verlierst du auch die Kontrolle über die Auswahl der Features.

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words, max_features=10000)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

## Achtung bei Bigrammen!

Wie bereits angesprochen können Bigramme den Feature-Space extrem aufblähen:

In [ ]:
count_vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,2), min_df=5)
count_vectors = count_vectorizer.fit_transform(df["nav"])
count_vectors

## Feature-Engineering ist wichtig

Das Feature-Eningeering und die Optimierung kannst du fast nicht überschätzen! Hier hast du einen tollen Hebel zur Verfügung, der dir 